# 元类
- 类是一组描述如何生成一个对象的代码断
- 在python中，类也是一种对象
- **创建类对象的类称为元类，即为type()**

```
- 内建模块builtins：默认加载（print...）
- globals()：查看已定义的变量
```

In [6]:
# 使用type()创建类,type(类名，继承/元组，方法属性/字典)
# type()即为元类

Test = type("Test",(),{"num":100,"num2":200})
print(Test)

<class '__main__.Test'>


In [15]:
# 在type中添加方法，带装饰器的方法同理

def test_1(self):
    print("这是实例方法1")

@staticmethod
def test_2():
    print("这是实例方法2")

Test2 = type("Test2",(),{"test_1":test_1,"test_2":test_2})

t2 = Test2()
t2.test_1()
t2.test_2()

这是实例方法1
这是实例方法2


In [17]:
# 查看是谁创建了类

class T:
    pass
t = T()
print(t.__class__)
print(t.__class__.__class__)
print(t.__class__.__class__.__class__)

<class '__main__.T'>
<class 'type'>
<class 'type'>


In [31]:
# 重写元类type()
# 可以写为函数，也可写为类
class Upper_attr(type):
    def __new__(cls,class_name,class_parents,class_attr):
        return type(class_name,class_parents,class_attr)

def upper_attr(class_name,class_parents,class_attr):
    new_attr = {}
    for name,value in class_attr.items():
        if not name.startswith("__"):
            new_attr[name.upper()] = value
            
    return type(class_name,class_parents,new_attr)

# 程序运行初始化时，类的创建由type()进行，当指定metaclass参数时，使用指定的函数创建类
class Foo(object, metaclass=upper_attr):
    bar = "bar"
    
# # python2中，如下写法：   
# class Foo2(object):
#     __metaclass__ = upper_attr
#     bar = "bar"
    
print(hasattr(Foo,"bar"))
print(hasattr(Foo,"BAR"))

f = Foo()
print(f.BAR)

False
True
bar


# 元类实现ORM
- orm：对象-关系映射
- 创建一个实例对象，用创建它的类名当做数据表名，用创建它的类属性当做字段，对数据的操作直接调用对象的方法

In [33]:
# 使用元类实现ORM
# 实现save方法

class ModelMetaclass(type):
    def __new__(cls, name, bases, attrs):
        mappings = dict()
        # 判断是否需要保存
        for k, v in attrs.items():
            # 判断是否是指定的StringField或者IntegerField的实例对象
            if isinstance(v, tuple):
                print('Found mapping: %s ==> %s' % (k, v))
                mappings[k] = v

        # 删除这些已经在字典中存储的属性
        for k in mappings.keys():
            attrs.pop(k)

        # 将之前的uid/name/email/password以及对应的对象引用、类名字
        attrs['__mappings__'] = mappings  # 保存属性和列的映射关系
        attrs['__table__'] = name  # 假设表名和类名一致
        return type.__new__(cls, name, bases, attrs)


class User(metaclass=ModelMetaclass):
    uid = ('uid', "int unsigned")
    name = ('username', "varchar(30)")
    email = ('email', "varchar(30)")
    password = ('password', "varchar(30)")
    # 当指定元类之后，以上的类属性将不在类中，而是在__mappings__属性指定的字典中存储
    # 以上User类中有 
    # __mappings__ = {
    #     "uid": ('uid', "int unsigned")
    #     "name": ('username', "varchar(30)")
    #     "email": ('email', "varchar(30)")
    #     "password": ('password', "varchar(30)")
    # }
    # __table__ = "User"
    def __init__(self, **kwargs):
        for name, value in kwargs.items():
            setattr(self, name, value)

    def save(self):
        fields = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v[0])
            args.append(getattr(self, k, None))

        args_temp = list()
        for temp in args:
            # 判断入如果是数字类型
            if isinstance(temp, int):
                args_temp.append(str(temp))
            elif isinstance(temp, str):
                args_temp.append("""'%s'""" % temp)
        sql = 'insert into %s (%s) values (%s)' % (self.__table__, ','.join(fields), ','.join(args_temp))
        print('SQL: %s' % sql)


u = User(uid=12345, name='Michael', email='test@orm.org', password='my-pwd')
# print(u.__dict__)
u.save()

Found mapping: uid ==> ('uid', 'int unsigned')
Found mapping: name ==> ('username', 'varchar(30)')
Found mapping: email ==> ('email', 'varchar(30)')
Found mapping: password ==> ('password', 'varchar(30)')
SQL: insert into User (uid,username,email,password) values (12345,'Michael','test@orm.org','my-pwd')


### 抽取基类

In [34]:
class ModelMetaclass(type):
    def __new__(cls, name, bases, attrs):
        mappings = dict()
        # 判断是否需要保存
        for k, v in attrs.items():
            # 判断是否是指定的StringField或者IntegerField的实例对象
            if isinstance(v, tuple):
                print('Found mapping: %s ==> %s' % (k, v))
                mappings[k] = v

        # 删除这些已经在字典中存储的属性
        for k in mappings.keys():
            attrs.pop(k)

        # 将之前的uid/name/email/password以及对应的对象引用、类名字
        attrs['__mappings__'] = mappings  # 保存属性和列的映射关系
        attrs['__table__'] = name  # 假设表名和类名一致
        return type.__new__(cls, name, bases, attrs)


class Model(object, metaclass=ModelMetaclass):
    def __init__(self, **kwargs):
        for name, value in kwargs.items():
            setattr(self, name, value)

    def save(self):
        fields = []
        args = []
        for k, v in self.__mappings__.items():
            fields.append(v[0])
            args.append(getattr(self, k, None))

        args_temp = list()
        for temp in args:
            # 判断入如果是数字类型
            if isinstance(temp, int):
                args_temp.append(str(temp))
            elif isinstance(temp, str):
                args_temp.append("""'%s'""" % temp)
        sql = 'insert into %s (%s) values (%s)' % (self.__table__, ','.join(fields), ','.join(args_temp))
        print('SQL: %s' % sql)


class User(Model):
    uid = ('uid', "int unsigned")
    name = ('username', "varchar(30)")
    email = ('email', "varchar(30)")
    password = ('password', "varchar(30)")


u = User(uid=12345, name='Michael', email='test@orm.org', password='my-pwd')
# print(u.__dict__)
u.save()

Found mapping: uid ==> ('uid', 'int unsigned')
Found mapping: name ==> ('username', 'varchar(30)')
Found mapping: email ==> ('email', 'varchar(30)')
Found mapping: password ==> ('password', 'varchar(30)')
SQL: insert into User (uid,username,email,password) values (12345,'Michael','test@orm.org','my-pwd')
